In [174]:
import torch
from torch import nn, optim
from torch.nn import functional as F

from torch.utils.data import DataLoader
import numpy as np
import sys
sys.path += ['..', '../..', '../../..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import Iterable, Tuple, List
from tqdm import tqdm
import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
from data_preparation import (
    Session, match_time, data_prep, sample_prep, stack_samples
)
sessions_v30Jun = Session.load_multiple_session('../../data/v30Jun')

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x62707f519680] moov atom not found


FrameMLToAngularVelocity not found
FrameMLToAngularVelocity not found


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x62707f642080] moov atom not found


In [181]:
sessions_v30Jun[0].data['angular_speed_control_df']

key,angle,angle_diff,angular_velocity,axis,degree_per_second,gyro_bias,initial_ori,left,linear_acc,new_proportion,q,right,speed,target_angle,time_AngularSpeedControl,time_OrientationTracker,warn
idx,,,,,,,,,,,,,,,,,
0,0.025025,0.0,0.803673,"[0.3234276954234821, 0.9459018776999164, 0.025...",0.0,"[-1.761709923664122, -1.0102595419847322, -0.9...","[0.6771594508496503, 0.035918289513238236, 0.7...",0.0,"[-0.95751953125, 0.052978515625, -0.08837890625]",0.502232,"[0.6770050755063988, 0.03596198153093591, 0.73...",0.0,0.0,0.025025,2024-06-29 09:42:46.652352,2024-06-29 09:42:46.641833,[]
1,0.324343,0.0,5.412713,"[-0.9842524715463302, 0.17524631448043296, 0.0...",0.0,None,None,0.0,"[-0.9609375, 0.042236328125, -0.080322265625]",0.515035,"[0.6768922464125785, 0.03398353070602194, 0.73...",0.0,0.0,0.324343,2024-06-29 09:42:46.706155,2024-06-29 09:42:46.704171,[]
2,0.388377,0.0,2.934704,"[-0.9749088566250654, 0.22112775684406952, 0.0...",0.0,None,None,0.0,"[-0.95654296875, 0.03662109375, -0.08251953125]",0.508152,"[0.6767234257668499, 0.03361685992952864, 0.73...",0.0,0.0,0.388377,2024-06-29 09:42:46.729107,2024-06-29 09:42:46.724852,[]
3,0.342558,0.0,-3.512627,"[-0.9454589762553158, 0.32303747068666006, 0.0...",0.0,None,None,0.0,"[-0.9580078125, 0.0478515625, -0.0830078125]",0.490243,"[0.6765482067944705, 0.033912234261649854, 0.7...",0.0,0.0,0.342558,2024-06-29 09:42:46.741202,2024-06-29 09:42:46.739555,[]
4,0.338574,0.0,-0.186142,"[-0.9115319515882644, 0.40816566522933834, 0.0...",0.0,None,None,0.0,"[-0.957275390625, 0.049560546875, -0.089111328...",0.499483,"[0.6763668923846887, 0.033985593980643755, 0.7...",0.0,0.0,0.338574,2024-06-29 09:42:46.762377,2024-06-29 09:42:46.759466,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635,194.993459,0.0,0.068404,"[0.002319317370084758, -0.01312370045044607, 0...",0.0,None,None,0.0,"[-0.958984375, 0.01708984375, -0.065185546875]",0.500190,"[-0.07886837522140717, -0.7317413575763336, -0...",0.0,0.0,194.993459,2024-06-29 09:44:18.578414,2024-06-29 09:44:18.577446,[]
8636,194.994628,0.0,0.118693,"[0.002532590886447981, -0.013226349971731858, ...",0.0,None,None,0.0,"[-0.960693359375, 0.0205078125, -0.068359375]",0.500330,"[-0.07880803484037024, -0.731596196547643, -0....",0.0,0.0,194.994628,2024-06-29 09:44:18.588269,2024-06-29 09:44:18.587394,[]
8637,194.995260,0.0,0.063133,"[0.0022797476795444246, -0.013265776635620617,...",0.0,None,None,0.0,"[-0.952392578125, 0.0234375, -0.08447265625]",0.500175,"[-0.07877401293934569, -0.7317656792679514, -0...",0.0,0.0,194.995260,2024-06-29 09:44:18.598286,2024-06-29 09:44:18.597419,[]


In [182]:
px.line(sessions_v30Jun[0].data['angular_speed_control_df'], x='time_AngularSpeedControl', y='degree_per_second')

In [135]:
from pytorch_train_tools import to_dataset, UpdatingPlotlyLines, to_float32, fit, temp_rnn_fit_step

In [6]:
model = Model()

In [140]:
for s in tqdm(sessions_v30Jun): data_prep(s), sample_prep(s)
x, y = stack_samples(sessions_v30Jun)
x = np.swapaxes(x, 1, -1)

100%|██████████| 2/2 [00:03<00:00,  1.50s/it]


In [145]:
x.shape

(6257, 3, 114, 64)

In [163]:
ds = to_windowed_dataset(x, y, 64)
loader = DataLoader(ds, shuffle=True, batch_size=None)


In [168]:
model = model.to('cuda')


In [164]:
for _x, _y in loader:
    _x,_y 

In [165]:
_x.shape

torch.Size([64, 3, 114, 64])

In [133]:
optimiser = optim.Adam(model.parameters(), lr=1e-4)


In [173]:
temp_rnn_fit_step(
    model, 
    loader, 
    optimiser, 
    F.huber_loss,
)

0.11779741268153734